In [ ]:
import pandas as pd
from src.grid import HexGrid
from src.h3_utils import *
from h3 import h3

## Import Predictions

In [ ]:
predictions = pd.read_pickle("~/Downloads/model_predictions_2020_7_16")
predictions.rename(columns={'stop_decr_lat':'latitude', 'stop_decr_lng':'longitude'}, inplace=True)

## Bin Predictions by H3 res=8 bins

In [ ]:
predictions['hour'] = predictions.end_datetime.apply(lambda x: x.hour)
predictions = predictions.reset_index().drop(columns=['trajectory_index', 'msa_name_start', 'end_datetime'])

hexbinned_predictions = bin_by_hexagon(predictions, groupby_items=['hex_id', 'hour'], agg_map={'prediction':'sum'}, resolution=8)

## Generate a hexagon for every hour and fill them (to include 0 predictions)

In [ ]:
filled_df = pd.DataFrame(hexbinned_predictions.hex_id.unique(), columns=['hex_id'])

In [ ]:
all_df = []
for i in range(25):
    filled_df_copy = filled_df.copy()
    filled_df_copy['hour'] = i
    filled_df_copy['prediction'] = 0
    all_df.append(filled_df_copy)
    
filled_df = pd.concat(all_df)

In [ ]:
for index, row in filled_df.iterrows():
    
    relevant_row = hexbinned_predictions[(hexbinned_predictions['hex_id'] == row['hex_id']) \
                          & (hexbinned_predictions['hour'] == row['hour'])]

    if not relevant_row.empty:
        filled_df.loc[index, 'prediction'] = relevant_row['prediction'].iloc[0]

In [ ]:
# Create geojson column
filled_df["geojson"] = filled_df.hex_id.apply(lambda x:
                                              {"type": "Polygon",
                                               "coordinates":
                                                   [h3.h3_to_geo_boundary(h=x, geo_json=True)]
                                               }
                                              )
filled_df.to_pickle('hexbinned_model_predictions.pkl')

In [ ]:
filled_df = pd.read_pickle('hexbinned_model_predictions.pkl')

In [ ]:
test = hexbinned_predictions.groupby('hex_id').agg({'prediction':'sum', 'geojson':'first'}).reset_index()

In [ ]:
test.rename(columns={'geojson':'geometry'}, inplace=True)

In [ ]:
hmap = h3_choropleth_map(test, value_to_map='prediction', kind='linear', hour=None)

In [ ]:
test[test.prediction == test.prediction.max()].geometry.iloc[0]

In [ ]:
hmap